https://www.analyticsvidhya.com/blog/2016/06/quick-guide-build-recommendation-engine-python/

#graphlab only compatible with python2,   
conda env 
C:\Users\spark>activate python27  
python27                 C:\Users\spark\AppData\Local\Continuum\Anaconda3\envs\python27

In [3]:
import os
os.getcwd()

'/Users/parksoy/Desktop/Soyoung_Udacity_ND_DeepLearning/embeddings/recommendation'

In [25]:
import pandas as pd
import IPython.display  

# pass in column names for each CSV and read them using pandas. 
# Column names available in the readme file

#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols, encoding='latin-1')

#Reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
         'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
         'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')

In [28]:
print("users", users.shape)
display(users.head(2))

print("ratings", ratings.shape)
display(ratings.head(2))

print("items", items.shape)
display(items.head(2))

users (943, 5)


,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043


ratings (100000, 4)


,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742


items (1682, 24)


,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [45]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')

print("ratings_base.shape=",ratings_base.shape, "ratings_test.shape=", ratings_test.shape)

display(ratings_base.head(2))

ratings_base.shape= (90570, 4) ratings_test.shape= (9430, 4)


,user_id,movie_id,rating,unix_timestamp
0,1,1,5,874965758
1,1,2,3,876893171


import turicreate as tc
#https://github.com/parksoy/turicreate

'''
##### Load data 
data = tc.SFrame('photoLabel.sframe')

##### Create a model
model = tc.image_classifier.create(data, target='photoLabel')

##### Make predictions
predictions = model.predict(data)

##### Export to Core ML
model.export_coreml('MyClassifier.mlmodel')
'''

In [30]:
import turicreate as tc

train_data = tc.SFrame(ratings_base)
test_data = tc.SFrame(ratings_test)

In [31]:
ratings_base.columns

Index(['user_id', 'movie_id', 'rating', 'unix_timestamp'], dtype='object')

In [35]:
train_data['user_id']

dtype: int
Rows: 90570
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... ]

In [10]:
popularity_model = tc.popularity_recommender.create(train_data, user_id='user_id', item_id='movie_id', target='rating')

Recsys training: model = popularity

Warning: Ignoring columns unix_timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 90570 observations with 943 users and 1680 items.

Data prepared in: 0.129537s

90570 observations to process; with 1680 unique items.

In [36]:
#Get recommendations for first 5 users and print them
#users = range(1,6) specifies user ID of first 5 users
#k=5 specifies top 5 recommendations to be given

popularity_recomm = popularity_model.recommend(users=train_data['user_id'][:5],k=5) #(range(1,6)) . user_id[:5]
#Query data contains column age, which was not present at train time.

popularity_recomm.print_rows(num_rows=25)

+---------+----------+-------+------+
| user_id | movie_id | score | rank |
+---------+----------+-------+------+
|    1    |   1467   |  5.0  |  1   |
|    1    |   1201   |  5.0  |  2   |
|    1    |   1189   |  5.0  |  3   |
|    1    |   1122   |  5.0  |  4   |
|    1    |   814    |  5.0  |  5   |
|    1    |   1467   |  5.0  |  1   |
|    1    |   1201   |  5.0  |  2   |
|    1    |   1189   |  5.0  |  3   |
|    1    |   1122   |  5.0  |  4   |
|    1    |   814    |  5.0  |  5   |
|    1    |   1467   |  5.0  |  1   |
|    1    |   1201   |  5.0  |  2   |
|    1    |   1189   |  5.0  |  3   |
|    1    |   1122   |  5.0  |  4   |
|    1    |   814    |  5.0  |  5   |
|    1    |   1467   |  5.0  |  1   |
|    1    |   1201   |  5.0  |  2   |
|    1    |   1189   |  5.0  |  3   |
|    1    |   1122   |  5.0  |  4   |
|    1    |   814    |  5.0  |  5   |
|    1    |   1467   |  5.0  |  1   |
|    1    |   1201   |  5.0  |  2   |
|    1    |   1189   |  5.0  |  3   |
|    1    | 

In [37]:
ratings_base.groupby(by='movie_id')['rating'].mean().sort_values(ascending=False).head(20)

movie_id
1500    5.000000
1293    5.000000
1122    5.000000
1189    5.000000
1656    5.000000
1201    5.000000
1599    5.000000
814     5.000000
1467    5.000000
1536    5.000000
1449    4.714286
1642    4.500000
1463    4.500000
1594    4.500000
1398    4.500000
114     4.491525
408     4.480769
169     4.476636
318     4.475836
483     4.459821
Name: rating, dtype: float64

In [39]:
#Train Model
item_sim_model = tc.item_similarity_recommender.create(train_data, \
                                                       user_id='user_id', \
                                                       item_id='movie_id', \
                                                       target='rating', \
                                                       similarity_type='pearson')


Recsys training: model = item_similarity

Warning: Ignoring columns unix_timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 90570 observations with 943 users and 1680 items.

...

| 316.775ms                           | 100              | 1680            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.338984s

In [41]:
#Make Recommendations:
item_sim_recomm = item_sim_model.recommend(users=train_data['user_id'][:5],k=5)
item_sim_recomm.print_rows(num_rows=25)

+---------+----------+-------+------+
| user_id | movie_id | score | rank |
+---------+----------+-------+------+
|    1    |   1467   |  5.0  |  1   |
|    1    |   1201   |  5.0  |  2   |
|    1    |   1189   |  5.0  |  3   |
|    1    |   1122   |  5.0  |  4   |
|    1    |   814    |  5.0  |  5   |
|    1    |   1467   |  5.0  |  1   |
|    1    |   1201   |  5.0  |  2   |
|    1    |   1189   |  5.0  |  3   |
|    1    |   1122   |  5.0  |  4   |
|    1    |   814    |  5.0  |  5   |
|    1    |   1467   |  5.0  |  1   |
|    1    |   1201   |  5.0  |  2   |
|    1    |   1189   |  5.0  |  3   |
|    1    |   1122   |  5.0  |  4   |
|    1    |   814    |  5.0  |  5   |
|    1    |   1467   |  5.0  |  1   |
|    1    |   1201   |  5.0  |  2   |
|    1    |   1189   |  5.0  |  3   |
|    1    |   1122   |  5.0  |  4   |
|    1    |   814    |  5.0  |  5   |
|    1    |   1467   |  5.0  |  1   |
|    1    |   1201   |  5.0  |  2   |
|    1    |   1189   |  5.0  |  3   |
|    1    | 

In [44]:
print(popularity_model, item_sim_model)

PopularityRecommender ItemSimilarityRecommender


In [42]:
model_performance = tc.compare(test_data, [popularity_model, item_sim_model])

tc.show_comparison(model_performance,[popularity_model, item_sim_model])

AttributeError: module 'turicreate' has no attribute 'compare'

# Text Classification

In [71]:
#https://github.com/kyle-kelly/reviewing_reviewers/blob/master/data/test_ratings_data.csv
data= pd.read_csv('ml-100k/rating-text.txt', sep=',', encoding='UTF-8')# latin-1 names=r_cols, 
data.IMDB=data.IMDB.astype(int)
data

,Titles,IMDB,Washington Post,The New York Times,The Telegraph,Los Angeles Times,Wall Street Journal,Chicago Tribune,Boston Globe,Rolling Stone,RogerEbert.com
0,Rogue One,7,63,50,80,70,40,75,63,88,88
1,Finding Dory,7,75,90,80,70,80,75,88,88,75
2,Captain America: Civil War,7,75,70,100,70,80,75,75,88,75
3,The Secret Life of Pets,6,50,60,60,30,40,-1,63,75,75
4,The Jungle Book,7,88,50,80,90,100,75,75,88,100
5,Deadpool,8,88,70,60,60,-1,63,75,75,50


In [72]:
#https://apple.github.io/turicreate/docs/userguide/text_classifier/
#https://apple.github.io/turicreate/docs/api/index.html

# Load data
#data = tc.SFrame('ratings_data.csv')
data=tc.SFrame(data)

# Create a model
#https://apple.github.io/turicreate/docs/api/generated/turicreate.sentence_classifier.create.html
model = tc.sentence_classifier.create(data, 'IMDB', features=['Titles'])

# Make predictions & evaluation the model
predictions = model.predict(data)
results = model.evaluate(data)

# Save the model for later use in Turi Create
model.save('MySentenceClassifier.model')

# Export for use in Core ML
#model.export_coreml('MySentenceClassifier.mlmodel')

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 6

Number of classes           : 3

...

| 1         | 2        | 0.000694     | 1.000000          |

| 2         | 3        | 0.002234     | 1.000000          |

| 3         | 4        | 0.002974     | 1.000000          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

In [74]:
classifier = model.classifier
print(classifier)

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 34
Number of examples             : 6
Number of classes              : 3
Number of feature columns      : 1
Number of unpacked features    : 16

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.2

Training Summary
----------------
Solver                         : newton
Solver iterations              : 3
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0035

Settings
--------
Log-likelihood                 : 0.529

Highest Positive Coefficients
-----------------------------
(intercept)                    : 2.2529
Titles[deadpool]               : 2.0085
(intercept)                    : 0.8802
Titles[book]                   : 0.6892
Titles[jungle]                 : 0.6892

Lowest Negative Coefficients
----------------------------
Titles[deadpool]               : -1.4067
Titles[pets]  

In [76]:
predictions

dtype: int
Rows: 6
[7, 7, 7, 6, 7, 8]

In [75]:
results

{'accuracy': 1.0, 'auc': 1.0, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 3
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      6       |        6        |   1   |
 |      7       |        7        |   4   |
 |      8       |        8        |   1   |
 +--------------+-----------------+-------+
 [3 rows x 3 columns], 'f1_score': 1.0, 'log_loss': 0.08816459956598961, 'precision': 1.0, 'recall': 1.0, 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 	class	int
 
 Rows: 300003
 
 Data:
 +-----------+-----+-----+---+---+-------+
 | threshold | fpr | tpr | p | n | class |
 +-----------+-----+-----+---+---+-------+
 |    0.0    | 1.0 | 1.0 | 1 | 5 |   0   |
 |   1e-05   | 1.0 | 1.0 | 1 | 5 |   0   |
 |   2e-05   | 1.0 | 1.0 | 1 | 5 |   0   |
 |   3e-05   | 1.0 | 1.0 | 1 | 5 |   0   |
 |   4e-05   | 1.0 | 1.0 | 1 | 5 |  

In [77]:
test_data= pd.read_csv('ml-100k/rating-text_testing.txt', sep=',', encoding='UTF-8')# latin-1 names=r_cols, 
test_data.IMDB=test_data.IMDB.astype(int)
test_data=tc.SFrame(test_data)
test_data

Titles,IMDB,Washington Post,The New York Times,The Telegraph,Los Angeles Times,Wall Street Journal
Zootopia,8,88,90,80,90,70
Batman v Superman: Dawnof Justice ...,6,37,30,40,70,20
Suicide Squad,6,37,60,40,50,0
Sing,7,63,50,40,70,-1
Moana,7,75,70,-1,80,80
Fantastic Beasts andWhere to Find Them ...,7,75,70,80,70,50
Doctor Strange,7,63,80,80,80,50
Hidden Figures,7,100,80,80,80,60
Jason Bourne,6,50,60,60,100,40
Star Trek Beyond,7,75,60,60,70,70


In [78]:
test_predictions = model.predict(test_data)
test_predictions

dtype: int
Rows: 19
[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]